# Loss Functions

In [0]:
from skimage import io
from skimage import transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

!wget https://upload.wikimedia.org/wikipedia/en/thumb/7/7d/Lenna_%28test_image%29.png/220px-Lenna_%28test_image%29.png

In [0]:
# Customized Network.
class CustomNetwork(nn.Module):
    
    def __init__(self, in_channels, num_classes=2):

        super(CustomNetwork, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 4, 3, 1, 1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2), # 256 -> 128

            nn.Conv2d(4, 8, 3, 1, 1),
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True), # 128 -> 64

            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(8, 4, kernel_size=2, stride=2, padding=0, output_padding=0), # 64 -> 128

            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(4, 1, kernel_size=2, stride=2, padding=0, output_padding=0), # 128 -> 256
        )
        
        self.initialize_weights()
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        
        print('x: ', x.size())
        
        # Feature extraction.
        x_hat = self.conv(x)
        print('x_hat: ', x_hat.size())
        
        return x_hat

In [0]:
def norm(img):
    
    mn = img.min()
    mx = img.max()
    
    img = (img - mn) / (mx - mn)
    
    return img

# Loading image, resizing to 256x256 and normalizing.
img = io.imread('220px-Lenna_(test_image).png')
io.imshow(img)
img = transform.resize(img, (256, 256)).astype(np.float32)
if len(img.shape) > 2:
    img = img[:,:,0]

img = norm(img)

print(img.shape)

In [0]:
# Transforming to tensor.
tensor = torch.from_numpy(img)
tensor = tensor.unsqueeze(0)
tensor = tensor.unsqueeze(0)
# tensor = tensor.cuda() # For GPU casting.

tensor = Variable(tensor, requires_grad=False)

print(tensor.size())
print(tensor)

In [0]:
import matplotlib.pyplot as plt

%matplotlib inline

# Instancing Network.
net = CustomNetwork(1, 2)
# net = CustomNetwork(1, 2).cuda() # For GPU casting.

# Instancing Loss.
#loss = nn.MSELoss()
loss = nn.L1Loss()

# Forwarding image.
output = net(tensor)
# output.requires_grad = False
output = output.detach() # For removing differentiation requirement.

# Computing loss.
rec_loss = loss(tensor, output)

print('reconstruction loss: ', rec_loss)

# Plotting images
fig, ax = plt.subplots(1, 2, figsize = (16, 8))

ax[0].imshow(tensor.numpy()[0, 0])
ax[0].set_title('Imagem Original')

ax[1].imshow(output.numpy()[0, 0])
ax[1].set_title('Imagem Reconstruída')

plt.show()

In [0]:
def custom_loss(inp, tar):
    return torch.mean(torch.abs(inp - tar))

# Instancing Network.
net = CustomNetwork(1, 2)
# net = CustomNetwork(1, 2).cuda() # For GPU casting.

# Forwarding image.
output = net(tensor)
# output.requires_grad = False
output = output.detach() # For removing differentiation requirement.

# Computing loss.
rec_loss = custom_loss(tensor, output)

print('reconstruction loss: ', rec_loss)

# Plotting images
fig, ax = plt.subplots(1, 2, figsize = (16, 8))

ax[0].imshow(tensor.numpy()[0, 0])
ax[0].set_title('Imagem Original')

ax[1].imshow(output.numpy()[0, 0])
ax[1].set_title('Imagem Reconstruída')

plt.show()